# 常规赛：PALM病理性近视预测基线方案

**赛题简述**

	PALM病理性近视预测常规赛的重点是研究和发展与病理性近视诊断相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务是将提供的图像分为病理性近视眼底彩照和非病理性近视眼底彩照，其中，非病理性近视眼底彩照包括正常眼底和高度近视眼底彩照。
    
![](https://ai-studio-static-online.cdn.bcebos.com/049a11b2a0d6459bbea817529d74be64b69ab7357c8f4990bc56a3b830aa6def)


**数据简介**

	PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带病理性近视分类标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。本次常规赛提供的病理性近视分类金标准是从临床报告中获取，不仅基于眼底彩照，还结合了OCT、视野检查等结果。

**数据基本标签**

    非病理：0
    病理  ：1

**训练数据集**

文件名称：Train

	Train文件夹里有一个fundus_image文件夹和一个Classification.xlsx文件。fundus_image文件夹中数据均为眼底彩照，分辨率为1444×1444，或2124×2056。命名形如N0001.jpg、H0001.jpg、P0001.jpg和V0001.jpg。Classification.xlsx文件中为各眼底图像是否属于病理性近视，属于为1，不属于为0。

**测试数据集**

文件名称：PALM-Testing400-Images 

	文件夹里包含400张眼底彩照，命名形如T0001.jpg。


In [ ]:
# 解压 常规赛：PALM病理性近视预测.zip
!unzip -oq data/data93479/常规赛：PALM病理性近视预测.zip -d dataset/
# 移除不必要的文件夹
!rm -rf dataset/__MACOSX

# 观察数据
## 观察图片 size
| Statistic | Width | Height |
| -------- | -------- | -------- |
| count	| 800.00000	| 800.000000	|
| mean	| 2033.90000	| 1974.910000	|
| std	| 230.68704	| 207.618336	|
| min	| 1444.00000	| 1444.000000	|
| 25%	| 2124.00000	| 2056.000000	|
| 50%	| 2124.00000	| 2056.000000	|
| 75%	| 2124.00000	| 2056.000000	|
| max	| 2124.00000	| 2056.000000	|
## 观察 labels 的分布
| label | count |
| -------- | -------- |
| 1	| 424	|
| 0	| 376	|


In [ ]:
"""
观察图片集中图片size的大小
"""
import os
from PIL import Image
import pandas as pd

train_img_dir = "dataset/常规赛：PALM病理性近视预测/Train/fundus_image"

size_list = []
for (dirpath, dirnames, filenames) in os.walk(train_img_dir):
    for filename in filenames:
        img_absolute_path = dirpath + "/" + filename
        image = Image.open(img_absolute_path)
        width, height = image.size
        size_list.append([filename, width, height])

image_size_df = pd.DataFrame(size_list, columns=['File', 'Width', 'Height'])

image_size_df.info()
image_size_df.describe()


In [ ]:
"""
读取xlsx文件，并转换为 image names list、image labels list
"""
import pandas as pd

# xlsx 文件，记录了图片名及其对应的标签
total_data_info = pd.read_excel('dataset/常规赛：PALM病理性近视预测/Train/Classification.xlsx')
total_data_info.info()

# Label 分类统计
print(total_data_info['Label'].value_counts())

# series dataframe 转 list
# total_data_info['imgName'].tolist()
total_data_list = total_data_info.values.tolist()

img_names = [x[0] for x in total_data_list]
img_labels = [x[1] for x in total_data_list]

print("image name length:", len(img_names))
print("image label length:", len(img_labels))


# 加载数据
## 图像增强
![](dataset/original_T0001.jpg)![](dataset/converted_T0001.jpg)
## 载入数据集
	from work.dataset import data_loader
	from work.dataset import TrainDataset
	# 训练时随机打乱数据顺序
	random.shuffle(filenames)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from paddle.vision.transforms import functional as F

img = Image.open("dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0001.jpg")
# img = cv2.imread("dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0001.jpg")
plt.imshow(img)
plt.savefig("dataset/original_T0001.jpg", dpi=100)
plt.show()

# 调节亮度
converted_img = F.adjust_brightness(img, 2)
plt.imshow(converted_img)
plt.savefig("dataset/converted_T0001.jpg", dpi=100)
plt.show()


In [ ]:
"""
yield 方式自定义的 data_loader, 适用于自定义的训练过程
"""
import pandas as pd

from work.dataset import data_loader

data_dir = 'dataset/常规赛：PALM病理性近视预测/Train/fundus_image'
total_data_info = pd.read_excel('dataset/常规赛：PALM病理性近视预测/Train/Classification.xlsx')
total_data_list = total_data_info.values.tolist()

train_loader = data_loader(data_dir, total_data_list, batch_size=50)
data_reader = train_loader()
image, label = next(data_reader)
# 查看数据类型
print("image_type:", type(image))
print("label_type:", type(label))
# 查看数据形状
image.shape, label.shape

In [ ]:
"""
使用 paddle API 写的 DataSet, DataLoader
适用于自动训练过程
"""
import pandas as pd
import paddle

from work.dataset import TrainDataset

data_dir = 'dataset/常规赛：PALM病理性近视预测/Train/fundus_image'
total_data_info = pd.read_excel('dataset/常规赛：PALM病理性近视预测/Train/Classification.xlsx')
total_data_list = total_data_info.values.tolist()

train_dataset = TrainDataset(data_dir, total_data_list)

print('=============custom dataset=============')
for data, label in train_dataset:
    print(data.shape, label.shape)
    break

# 加载数据
train_loader = paddle.io.DataLoader(train_dataset, return_list=True, shuffle=True, 
                                    batch_size=10, drop_last=True)
for data, label in train_loader:
    print(data.shape, label.shape)
    break


# 网络搭建

**飞桨框架内置模型**

	['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'VGG', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'MobileNetV1', 'mobilenet_v1', 'MobileNetV2', 'mobilenet_v2', 'LeNet']


In [ ]:
import paddle

print('飞桨框架内置模型：', paddle.vision.models.__all__)


In [ ]:
import paddle
from paddle.vision.models import resnet18

# build model
resnet18_model = resnet18(pretrained=True)

# paddle.summary(resnet18, (64, 3, 112, 112))

x = paddle.rand([1, 3, 224, 224])
out = resnet18_model(x)

print("resnet18 out_shape:", out.shape)

model = paddle.nn.Sequential(
    resnet18_model,
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(1000, 2),
    paddle.nn.Softmax()
)

paddle.summary(model, (64, 3, 360, 360))


In [1]:
import random
import paddle
from paddle.vision.models import resnet18, resnet34, resnet50
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.transforms import Transpose
from paddle.static import InputSpec

import pandas as pd
from work.dataset import TrainDataset

use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

data_dir = 'dataset/常规赛：PALM病理性近视预测/Train/fundus_image'
total_data_info = pd.read_excel('dataset/常规赛：PALM病理性近视预测/Train/Classification.xlsx')
total_data_list = total_data_info.values.tolist()

random.shuffle(total_data_list)
train_data_list = total_data_list[:600]
val_data_list = total_data_list[600:]

train_dataset = TrainDataset(data_dir, train_data_list, mode='train')
val_dataset = TrainDataset(data_dir, val_data_list, mode='val')

train_loader = paddle.io.DataLoader(train_dataset, return_list=True, shuffle=True, batch_size=100)
val_loader = paddle.io.DataLoader(val_dataset, return_list=True, batch_size=100)

print('=============custom dataset=============')
for data, label in train_loader:
    print("train shape:", data.shape, label.shape)
    break
for data, label in val_loader:
    print("val shape:", data.shape, label.shape)
    break

model_name = "resnet18"

if model_name == "resnet18":
    resnet_model = resnet18(pretrained=True)
elif model_name == "resnet34":
    resnet_model = resnet34(pretrained=True)
elif model_name == "resnet50":
    resnet_model = resnet50(pretrained=True)

# 包含 resnet 的 Sequential 模型
resnet_seq = paddle.nn.Sequential(
    resnet_model,
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(1000, 2),
    paddle.nn.Softmax()
)

# 调用 resnet50 模型
# model_input = InputSpec([-1, 3, 360, 360], 'float32', 'image')
# model = paddle.Model(resnet50(pretrained=True, num_classes=2), model_input)

model = paddle.Model(resnet_seq)

# 定义优化器
optimizer = paddle.optimizer.Adam(learning_rate=1e-4, parameters=model.parameters())
# 进行训练前准备
model.prepare(optimizer, CrossEntropyLoss(), Accuracy())
callback = paddle.callbacks.VisualDL(log_dir='log/visualdl_log_dir/{}_output_adjust_brightness'.format(model_name))

print('============= start fit =============')
# 启动训练
model.fit(train_loader,
          val_loader,
          epochs=20,
          save_dir="./log/checkpoints/{}_output_adjust_brightness".format(model_name),
          callbacks=callback)


=============custom dataset=============


In [ ]:
eval_result = model.evaluate(val_loader, verbose=1)

In [ ]:
pre_result = model.predict(train_loader)


In [ ]:
import os
import cv2
import paddle

from work.dataset import transform_img

test_dir = "dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images"
class TestDataset(paddle.io.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.test_img_names = os.listdir(data_dir)

    def __getitem__(self, idx):
        file_name = self.test_img_names[idx]
        filepath = os.path.join(self.data_dir, file_name)
        img = cv2.imread(filepath)
        data = transform_img(img)
        return data
    
    def __len__(self):
        return len(self.test_img_names)

test_dataset = TestDataset(test_dir)

model.load('log/checkpoints/resnet34_output/8.pdparams')

result = model.predict(test_dataset, batch_size=100)


In [ ]:
import numpy as np
print(type(result))
# print(result)
result_np = np.array(result)
print(result_np.shape)
result_np = result_np.reshape((-1, 2))
print(result_np.shape)
result_sub = np.argmax(result_np, axis=1)
print(type(result_sub), result_sub.shape)
test_img_names = np.array(os.listdir(test_dir))
print(type(test_img_names), test_img_names.shape)
submission_np = np.concatenate((np.expand_dims(test_img_names, 0), np.expand_dims(result_sub, 0)), axis=0).transpose()

In [ ]:
import pandas as pd
Submit_data = pd.DataFrame(submission_np)          # 转为DataFrame格式
Submit_data.columns = ['FileName', 'PM Risk']    # 修改列名
Submit_data = Submit_data.sort_values(by='FileName').reset_index(drop=True)   # 按照图片id排列
Submit_data.to_csv('Classification_Results.csv', index=False, float_format="%.1f")       # 保存结果csv

In [ ]:
import os
import cv2
import numpy as np
import paddle
from paddle.vision.models import resnet50
from paddle.static import InputSpec

from work.dataset import transform_img

test_dir = "dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images"
class TestDataset(paddle.io.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.test_img_names = os.listdir(data_dir)

    def __getitem__(self, idx):
        file_name = self.test_img_names[idx]
        filepath = os.path.join(self.data_dir, file_name)
        img = cv2.imread(filepath)
        data = transform_img(img)
        return data
    
    def __len__(self):
        return len(self.test_img_names)

test_dataset = TestDataset(test_dir)

# 定义数据读取器
def test_data_loader(data_dir, batch_size=10):
    test_image_names = os.listdir(data_dir)
    def reader():
        batch_imgs = []
        for i in range(len(test_image_names)):
            test_image_name = test_image_names[i]
            filepath = os.path.join(data_dir, test_image_name)
            img = cv2.imread(filepath)
            img = transform_img(img)
            # 每读取一个样本的数据，就将其放入数据列表中
            batch_imgs.append(img)
            if len(batch_imgs) == batch_size:
                # 当数据列表的长度等于batch_size的时候，
                # 把这些数据当作一个mini-batch，并作为数据生成器的一个输出
                imgs_array = np.array(batch_imgs).astype('float32')
                yield imgs_array
                batch_imgs = []

        if len(batch_imgs) > 0:
            # 剩余样本数目不足一个batch_size的数据，一起打包成一个mini-batch
            imgs_array = np.array(batch_imgs).astype('float32')
            yield imgs_array

    return reader

test_loader = test_data_loader(test_dir, batch_size=100)
test_data_reader = test_loader()

# 调用resnet50模型
model_input = InputSpec([-1, 3, 360, 360], 'float32', 'image')
model = paddle.Model(resnet50(pretrained=True, num_classes=2), model_input)

model.load('output/6.pdparams')
print('============= start predict =============')

result = model.predict(test_dataset, batch_size=100)


# 后话

本项目构建了一个可以使用各类经典卷积神经网络进行图像二分类的模型，并采用了图像增强的方法（项目中仅使用了亮度调整 ）。

因为当时的目的仅作为练习使用，所以没有刻意多次调参追求模型效果。

上次结果提交距离这次提交审查也有一个月时间，所以不太清楚提交的结果是哪一次的 checkpoint。

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 